# Extract publication times

More processing and filtering of this data might be needed. Useful reference from a past analysis is at https://nbviewer.jupyter.org/github/dhimmel/delays/blob/upgrade/3.extract-delays.ipynb

In [1]:
import pandas
import re

In [2]:
date_cols = ["published", "accepted_0", "received_0", "retracted_0"]
date_df = pandas.read_table("data/litcovid-article-dates.tsv", parse_dates=date_cols)
date_df.head(2)

,pubmed_id,journal_nlm_id,journal,doi,epub,pub,publication_types,published,accepted_0,aheadofprint_0,entrez_0,medline_0,pmc-release_0,pubmed_0,received_0,retracted_0,revised_0,revised_1,version_0,title
0,31943059,9434456,J Travel Med,10.1093/jtm/taaa008,NaN,2020-03-13,Journal Article,2020-03-13,2020-01-10,NaN,2020-01-17,2020-03-21,NaN,2020-01-17,2020-01-08,NaT,2020-01-09,NaN,NaN,"Pneumonia of unknown aetiology in Wuhan, China..."
1,31944312,7705876,J Med Virol,10.1002/jmv.25674,2020-02-04,NaN,Journal Article | Review,2020-02-04,2020-01-12,NaN,2020-01-17,2020-03-20,NaN,2020-01-17,2019-12-23,NaT,NaN,NaN,NaN,Recent advances in the detection of respirator...


In [3]:
date_df["days_received_to_published"] = (date_df.published - date_df.received_0).dt.days
date_df["days_received_to_accepted"] = (date_df.accepted_0 - date_df.received_0).dt.days
date_df["days_accepted_to_published"] = (date_df.published - date_df.accepted_0).dt.days

In [4]:
date_df = (
    date_df
    .reindex(columns=[
        "pubmed_id",
        "journal",
        "doi",
        "published",
        "days_received_to_published",
        "days_received_to_accepted",
        "days_accepted_to_published",
        "publication_types",
        "title",
    ])
    .convert_dtypes()
)
date_df.head(3)

,pubmed_id,journal,doi,published,days_received_to_published,days_received_to_accepted,days_accepted_to_published,publication_types,title
0,31943059,J Travel Med,10.1093/jtm/taaa008,2020-03-13,65,2,63,Journal Article,"Pneumonia of unknown aetiology in Wuhan, China..."
1,31944312,J Med Virol,10.1002/jmv.25674,2020-02-04,43,20,23,Journal Article | Review,Recent advances in the detection of respirator...
2,31950516,J Med Virol,10.1002/jmv.25678,2020-02-12,30,2,28,Journal Article,Outbreak of pneumonia of unknown etiology in W...


In [5]:
date_df.to_csv("data/litcovid-article-times.tsv", sep="\t", index=False)